<a href="https://colab.research.google.com/github/thimotyb/real-world-machine-learning/blob/python3/ReinforcementLearning_NChain_OpenGym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning
## Open-AI Gym n-chain example with neural networks
[link text](https://adventuresinmachinelearning.com/reinforcement-learning-tutorial-python-keras/)

The NChain example on Open AI Gym is a simple 5 state environment. There are two
possible actions in each state, move forward (action 0) and move backwards (action 1).
When action 1 is taken, i.e. move backwards, there is an immediate reward of 2 given to
the agent – and the agent is returned to state 0 (back to the beginning of the chain).
However, when a move forward action is taken (action 0), there is no immediate reward
until state 4. When the agent moves forward while in state 4, a reward of 10 is received by
the agent. The agent stays in state 4 at this point also, so the reward can be repeated.
There is also a random chance that the agent’s action is “flipped” by the environment (i.e.
an action 0 is flipped to an action 1 and vice versa). The diagram below demonstrates this
environment:

![alt text](https://adventuresinmachinelearning.com/wp-content/uploads/2018/02/NChain-illustration.png)

Reinforcement learning – the basics
Reinforcement learning can be considered the third genre of the machine learning triad – unsupervised learning, supervised learning and reinforcement learning. In supervised learning, we supply the machine learning system with curated (x, y) training pairs, where the intention is for the network to learn to map x to y. In reinforcement learning, we create an agent which performs actions in an environment and the agent receives various rewards depending on what state it is in when it performs the action. In other words, an agent explores a kind of game, and it is trained by trying to maximize rewards in this game. This cycle is illustrated in the figure below:

![alt text](https://i2.wp.com/adventuresinmachinelearning.com/wp-content/uploads/2018/02/Reinforcement-learning-environment.png?w=381&ssl=1)

As can be observed above, the agent performs some action in the environment. An interpreter views this action in the environment, and feeds back an updated state that the agent now resides in, and also the reward for taking this action. The environment is not known by the agent beforehand, but rather it is discovered by the agent taking incremental steps in time. So, for instance, at time t the agent, in state ,  may take action a. This results in a new state  and a reward r. This reward can be a positive real number, zero, or a negative real number. It is the goal of the agent to learn which state dependent action to take which maximizes its rewards. The way which the agent optimally learns is the subject of reinforcement learning theory and methodologies.

To more meaningfully examine the theory and possible approaches behind reinforcement learning, it is useful to have a simple example in which to work through. This simple example will come from an environment available on Open AI Gym called NChain.

## Playing with Gym

The step() command returns 4 variables in a tuple, these are (in order):

The new state after the action
The reward due to the action
Whether the game is “done” or not – the NChain game is done after 1,000 steps
Debugging information – not relevant in this example

In [11]:
# Prepare the Gym
import gym
env = gym.make('NChain-v0')
env.reset()

0

In [12]:
# Simulate a few steps of the game
print(env.step(1))
print(env.step(0))
print(env.step(0))
print(env.step(0))
print(env.step(0))
print(env.step(0))
print(env.step(0))
print(env.step(0))
print(env.step(0)) # Reaches the max reward in the game
print(env.step(0))
print(env.step(0))

(0, 2, False, {})
(1, 0, False, {})
(2, 0, False, {})
(3, 0, False, {})
(4, 0, False, {})
(0, 2, False, {})
(1, 0, False, {})
(2, 0, False, {})
(3, 0, False, {})
(4, 0, False, {})
(4, 10, False, {})


As can be observed, starting in state 0 and taking step(1) action, the agent stays in state 0 and gets 2 for its reward. Next, I sent a series of action 0 commands. After every action 0 command, we would expect the progression of the agent along the chain, with the state increasing in increments (i.e. 0 -> 1 -> 2 etc.). However, you’ll observe after the first step(0) command, that the agent stays in state 0 and gets a 2 reward. This is because of the random tendency of the environment to “flip” the action occasionally, so the agent actually performed a 1 action. This is just unlucky.

Nevertheless, I persevere and it can be observed that the state increments as expected, but there is no immediate reward for doing so for the agent until it reaches state 4. When in state 4, an action of 0 will keep the agent in step 4 and give the agent a 10 reward. Not only that, the environment allows this to be done repeatedly, as long as it doesn’t produce an unlucky “flip”, which would send the agent back to state 0 – the beginning of the chain.

# A first naive heuristic for reinforcement learning

In order to train the agent effectively, we need to find a good policy pi which maps states to actions in an optimal way to maximize reward. There are various ways of going about finding a good or optimal policy, but first, let’s consider a naive approach.

Let’s conceptualize a table, and call it a reward table, which looks like this:



$\begin{bmatrix}r_{s_0,a_0} & r_{s_0,a_1}\\ r_{s_1,a_0} & r_{s_1,a_1} \\
r_{s_2,a_0} & r_{s_2,a_1} \\
r_{s_3,a_0} & r_{s_3,a_1} \\
r_{s_4,a_0} & r_{s_4,a_1} \end{bmatrix}$



Each of the rows corresponds to the 5 available states in the NChain environment, and each column corresponds to the 2 available actions in each state – forward and backward, 0 and 1. The value in each of these table cells corresponds to some measure of reward that the agent has “learnt” occurs when they are in that state and perform that action. So, the value  would be, say, the sum of the rewards that the agent has received when in the past they have been in state 0 and taken action 0. This table would then let the agent choose between actions based on the summated (or average, median etc. – take your pick) amount of reward the agent has received in the past when taking actions 0 or 1.

This might be a good policy – choose the action resulting in the greatest previous summated reward. Let’s give it a try, the code looks like:

In [0]:
def naive_sum_reward_agent(env, num_episodes=500):
    # this is the table that will hold our summated rewards for
    # each action in each state
    r_table = np.zeros((5, 2))
    for g in range(num_episodes):
        s = env.reset()
        done = False
        while not done:
            if np.sum(r_table[s, :]) == 0:
                # make a random selection of actions
                a = np.random.randint(0, 2)
            else:
                # select the action with highest cummulative reward
                a = np.argmax(r_table[s, :])
            new_s, r, done, _ = env.step(a)
            r_table[s, a] += r
            s = new_s
    return r_table